# Island defect on 

## 0. Introduction

This tutorial demonstrates the process of creating a rectangular island defect on a slab. 


> **D. G. Sangiovanni, A. B. Mei, D. Edström, L. Hultman, V. Chirita, I. Petrov, and J. E. Greene**, "Effects of surface vibrations on interlayer mass transport: Ab initio molecular dynamics investigation of Ti adatom descent pathways and rates from TiN/TiN(001) islands", Physical Review B, 2018. [DOI: 10.1103/PhysRevB.97.035406](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.97.035406).
> 
Below is the figure 1 from the manuscript demonstrating the nitrogen substitution defects in graphene.

![Fig 2.](https://journals.aps.org/prb/article/10.1103/PhysRevB.97.035406/figures/2/medium)

In this notebook we will reproduce material from FIG. 2 a)


## 1. Prepare the Environment
### 1.1. Set up defects parameters 



In [53]:
# Choose the island shape: 'cylinder', 'sphere', 'box', or 'triangular_prism'
ISLAND_SHAPE = 'box'
AUTO_ADD_VACUUM = True
VACUUM_THICKNESS = 10.0

BOX_PARAMS = {
    'min_coordinate': [0.25, 0.25, 0],
    'max_coordinate': [0.75, 0.75, 1]
}

# Create slab with default parameters if imported material is not a slab
DEFAULT_SLAB_PARAMETERS = {
    "miller_indices": (0,0,1),
    "thickness": 3,
    "vacuum": 5.0,
    "use_orthogonal_z": True,
    "xy_supercell_matrix": [[9, 0], [0, 9]]
}

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [54]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("", "../../config.yml")

### 1.3. Get input material
Materials are loaded with `get_materials()`.

In [55]:
from utils.jupyterlite import load_material_from_folder

# TODO: load from standata
material = load_material_from_folder("../uploads", "Ti4 N4")

INFO: 0: 0-Ni
INFO: 1: 1-Graphene
INFO: 2: 10-Al2O3, Sapphire, RHL (R-3c) 3D (Bulk), mp-1143 (1)
INFO: 3: 11-ZnO, Zinc Oxide, HEX (P6_3mc) 3D (Bulk), mp-2133
INFO: 4: 12-Cd4 Te4
INFO: 5: 13-Si4 C4
INFO: 6: 14-GaN, Gallium Nitride, HEX (P6_3mc) 3D (Bulk), mp-804
INFO: 7: 15-WS2, Tungsten Disulfide, HEX (P-6m2) 2D (Monolayer), 2dm-3749
INFO: 8: 16 - In4 P4
INFO: 9: 17 - Ti4 N4
INFO: 10: 4-Te2Mo
INFO: 11: 5-HfO2
INFO: 12: 6-Ni4(110), termination Ni_Pmmm_2, Slab, Terrace, 1 steps, [2 0 0]
INFO: 13: 7-Ag4
INFO: 14: 8-Si, Silicene, HEX (P-3m1) 2D (Monolayer), 2dm-5934
INFO: 15: 9-GaAs, Gallium Arsenide, FCC (F-43m) 3D (Bulk), mp-2534
INFO: 16: C1, Vacancy  Defect
INFO: 17: Cu1 Ni27, Interstitial Cu Defect
INFO: 18: Ga2N2(001)-C4Si4(111), Interface, Strain 0.082pct
INFO: 19: Heterostructure_M0_M1_M2
INFO: 20: N1, Vacancy  Defect
INFO: 21: Ni26 C1, Substitution C Defect
INFO: 22: Ni27 C1, Interstitial C Defect
INFO: 23: Ni4(111), termination Ni_P6:mmm_4, Slab
INFO: 24: Ni4(111), termination Ni

### 1.4. Preview Slab or create a new one

In [56]:
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration
from utils.visualize import visualize_materials as visualize

slab = material
if not slab.metadata or slab.metadata["build"]["configuration"]["type"] != SlabConfiguration.__name__:
    print("The material is not a slab. Creating a new slab...")
    slab_config = SlabConfiguration(
        bulk=material,
        miller_indices=DEFAULT_SLAB_PARAMETERS["miller_indices"],
        thickness=DEFAULT_SLAB_PARAMETERS["thickness"],
        vacuum= DEFAULT_SLAB_PARAMETERS["vacuum"],
        use_orthogonal_z=DEFAULT_SLAB_PARAMETERS["use_orthogonal_z"],
        xy_supercell_matrix=DEFAULT_SLAB_PARAMETERS["xy_supercell_matrix"]
    )
        
    slab = create_slab(slab_config)

visualize([{"material": slab, "title": "Original material"}])
visualize([{"material": slab, "title": "Original material"}], rotation="-90x")

The material is not a slab. Creating a new slab...


GridBox(children=(VBox(children=(Label(value='N972Ti972 - Original material - rotation: 0x,0y,0z', layout=Layo…

GridBox(children=(VBox(children=(Label(value='N972Ti972 - Original material - rotation: -90x', layout=Layout(a…

## 2. Create the Target Material
### 2.1. Initialize the defect configuration and builder

In [57]:
from mat3ra.made.tools.utils.coordinate import (
    BoxCoordinateCondition,
)
from mat3ra.made.tools.build.defect import IslandSlabDefectConfiguration

coordinate_condition= BoxCoordinateCondition(
    min_coordinate=BOX_PARAMS['min_coordinate'],
    max_coordinate=BOX_PARAMS['max_coordinate']
)


island_config = IslandSlabDefectConfiguration(
    crystal=slab,
    condition=coordinate_condition,
    number_of_added_layers=1,
)

from mat3ra.made.tools.build.defect import IslandSlabDefectBuilder, SlabDefectBuilderParameters
params = SlabDefectBuilderParameters(
    auto_add_vacuum=AUTO_ADD_VACUUM,
    vacuum_thickness=VACUUM_THICKNESS,
)
builder = IslandSlabDefectBuilder(build_parameters=params)

### 2.2. Create the island

In [58]:
slab_with_island = builder.get_material(island_config)

## 3. Visualize the Result(s)

In [59]:
visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": f"Material with Island Defect ({ISLAND_SHAPE})"}],
rotation="-90x")

visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": f"Material with Island Defect ({ISLAND_SHAPE})"}])

GridBox(children=(VBox(children=(Label(value='N972Ti972 - Original material - rotation: -90x', layout=Layout(a…

GridBox(children=(VBox(children=(Label(value='N972Ti972 - Original material - rotation: 0x,0y,0z', layout=Layo…

## 4. Pass data to the outside runtime

In [60]:
from utils.jupyterlite import set_materials

set_materials(slab_with_island)

Data for materials written to uploads/N4Ti4(001), termination TiN_P4:mmm_4, Slab.json
